# Example: Setting up and solving a matrix-style LP with Python/Gurobi
## [Jon Lee](https://sites.google.com/site/jonleewebpage/home), 10/20/2020

\begin{align*}
& \min c'x+f'w\\
& Ax+Bw \leq b\\
& Dx \phantom{+Bw~} = g\\
&x\geq 0,~ w\leq 0
\end{align*}

Note that we have the following dual, but we don't model it:

\begin{align*}
& \max y'b + v'g\\
& y'A + v'D \leq c'\\
& y'B \phantom{+v'D~} \geq f'\\
&y\leq 0,~ v \hbox{ unrestricted}
\end{align*}

Rather, we recover its solution from Gurobi.

References:
* Jon Lee, "A First Course in Linear Optimization", Fourth Edition (Version 4.0), Reex Press, 2013-20.

MIT License

Copyright (c) 2020 Jon Lee

Permission is hereby granted, free of charge, to any person obtaining a copy 
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
%reset -f
import numpy as np
import gurobipy as gp
from gurobipy import GRB

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [2]:
# setting the matrix sizes and random data
n1=7
n2=15
m1=2
m2=4
np.random.seed(56) # set seed to be able to repeat the same random data
A=np.random.rand(m1,n1)
B=np.random.rand(m1,n2)
D=np.random.rand(m2,n1)

# Organize the situation (i.e., choose the right-hand side coefficients) 
#   so that the primal problem has a feasible solution
xs=np.random.rand(n1)
ws=-np.random.rand(n2)
b=np.matmul(A,xs)+np.matmul(B,ws)+0.01*np.random.rand(m1)
g=np.matmul(D,xs)

# Organize the situation (i.e., choose the objective coefficients) 
#   so that the dual problem has a feasible solution
ys=-np.random.rand(m1)
vs=np.random.rand(m2)-np.random.rand(m2)
c=np.matmul(np.transpose(A),ys)+np.matmul(np.transpose(D),vs)+0.01*np.random.rand(n1)
f=np.matmul(np.transpose(B),ys)-0.01*np.random.rand(n2)

In [3]:
model = gp.Model()
x = model.addMVar(n1)   # default is a nonnegative continuous variable
w = model.addMVar(n2, ub=0.0, lb=-GRB.INFINITY)
objective = model.setObjective(c@x+f@w, GRB.MINIMIZE)
constraints1 = model.addConstr(A@x+B@w <= b)
constraints2 = model.addConstr(D@x == g)

Academic license - for non-commercial use only - expires 2021-08-26
Using license file C:\Users\jonxlee\gurobi.lic
Discarded solution information


In [4]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution")
    raise StopExecution

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 22 columns and 72 nonzeros
Model fingerprint: 0x734450bc
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 3e+00]
Presolve time: 0.01s
Presolved: 6 rows, 22 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.2547823e+30   1.946982e+31   7.254782e+00      0s
       9    2.6453973e+00   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.02 seconds
Optimal objective  2.645397253e+00


In [5]:
print("***** Primal solution:")
for j in range(0,n1): print("x[",j,"]=",
    np.format_float_positional(np.ndarray.item(x[j].X),4,pad_right=4))
print(" ")
for j in range(0,n2): print("w[",j,"]=",
    np.format_float_positional(np.ndarray.item(w[j].X),4,pad_right=4))
print(" ") 
print("***** Dual solution:")
for i in range(0,m1): print("y[",i,"]=",
    np.format_float_positional(constraints1[i].Pi,4,pad_right=4))
print(" ")
for i in range(0,m2): print("v[",i,"]=",
    np.format_float_positional(constraints2[i].Pi,4,pad_right=4))

***** Primal solution:
x[ 0 ]= 0.2689
x[ 1 ]= 0.0080
x[ 2 ]= 1.3952
x[ 3 ]= 0.    
x[ 4 ]= 0.4962
x[ 5 ]= 0.    
x[ 6 ]= 0.    
 
w[ 0 ]= 0.    
w[ 1 ]= 0.    
w[ 2 ]= 0.    
w[ 3 ]= 0.    
w[ 4 ]= 0.    
w[ 5 ]= 0.    
w[ 6 ]= 0.    
w[ 7 ]= 0.    
w[ 8 ]= 0.    
w[ 9 ]= 0.    
w[ 10 ]= -4.7348
w[ 11 ]= 0.    
w[ 12 ]= -4.392 
w[ 13 ]= 0.    
w[ 14 ]= 0.    
 
***** Dual solution:
y[ 0 ]= -0.4424
y[ 1 ]= -0.7261
 
v[ 0 ]= -0.8196
v[ 1 ]= -0.6668
v[ 2 ]= -0.0458
v[ 3 ]= 0.1904
